<a href="https://colab.research.google.com/github/Shakkyra/TC3004B/blob/main/Actividad2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 2.1: Redes Neuronales Entrenadas
- Liliana Solórzano Perez A01641392
- Diego Sebastián García Cabrera A01634071

## Instrucciones
Investiga un ejemplo de una red neuronal artificial en la literatura, la cual ya este entrenada, es decir, es un modelo completamente funcional y que se pueda utilizar una vez se tenga descargada, por ejemplo: redes para reconocimiento de objetos en imágenes (MobileNet) o redes para reconocimiento de palabras (SpeechCommands18w de ML5).

Crea una aplicacion simple ya sea en Python o JavaScript que pueda interactual con el modelo de red neuronal previamente descargado.

Realiza pruebas para ver el funcionamiento de la red neuronal.

## Resumen
Este código utiliza una red neuronal llamada YAMNet para clasificar sonidos.
Graba audio desde tu micrófono, lo procesa y luego intenta identificar qué tipo de sonido es (como voz, música, ruido, etc.).
Los resultados se muestran con la categoría predicha y la confianza del modelo.

## Pasos a seguir
Pasos:

1. Ejecutar todas las celdas de código: Comenzando desde la primera celda (la que contiene las importaciones e instalaciones) hasta la última (la que llama a la función main()). Puedes hacer clic en el botón de reproducción junto a cada celda o usar el menú "Entorno de ejecución" para ejecutar todas las celdas.
2. Permitir acceso al micrófono: Cuando se te solicite, otorga acceso a Colab a tu micrófono. Esto es necesario para grabar audio.
3. Grabar tu audio: Cuando veas el mensaje "Grabando durante 3 segundos...", habla o haz un sonido en tu micrófono durante 3 segundos. La grabación se detendrá automáticamente.
4. Esperar el procesamiento: Colab procesará el audio grabado y lo ejecutará a través del modelo YAMNet.
5. Ver los resultados: La celda de salida mostrará la categoría predicha del sonido (por ejemplo, "speech", "music", "noise") y el nivel de confianza del modelo. También verás predicciones más detalladas para cada fragmento del audio.


In [ ]:
# Step 1
!apt-get install -y portaudio19-dev python3-pyaudio
!pip install sounddevice soundfile
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

!pip install PyAudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  python3-pyaudio
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 25.9 kB of archives.
After this operation, 117 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 25.9 kB in 0s (83.0 kB/s)
Selecting previously unselected package python3-pyaudio.
(Reading database ... 126376 files and directories currently installed.)
Preparing to unpack .../python3-pyaudio_0.2.11-1.3ubuntu1_amd64.deb ...
Unpacking python3-pyaudio (0.2.11-1.3ubuntu1) ...
Setting up python3-pyaudio (0.2.11-1.3ubuntu1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the

In [ ]:
# Step 2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
from IPython.display import Javascript, Audio, display
from base64 import b64decode
import io
import os
import requests

# Instalar dependencias necesarias
!pip install soundfile tensorflow_hub

# Descargar el modelo YAMNet
model_url = "https://tfhub.dev/google/yamnet/1"
model = hub.load(model_url)

# Descargar las clases de YAMNet
class_map_url = "https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv"
class_map_path = "yamnet_class_map.csv"

# Descargar el mapeo de clases
print("Descargando el mapeo de clases de YAMNet...")
response = requests.get(class_map_url)
with open(class_map_path, 'wb') as f:
    f.write(response.content)

# Cargar las clases
class_names = []
with open(class_map_path, 'r') as f:
    next(f)  # Saltar la cabecera
    for line in f:
        class_names.append(line.strip().split(',')[2])

print(f"Cargadas {len(class_names)} clases.")

# Comandos soportados (mapeados a clases YAMNet)
COMMANDS = {
    'Speech': 'speech',
    'Silence': 'silence',
    'Noise': 'noise',
    'Music': 'music',
    'Child speech': 'child_speech',
    'Male speech': 'male_speech',
    'Female speech': 'female_speech'
}

# Función JavaScript mejorada para grabar audio (con duración fija de 3 segundos)
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

async function recordAudio(time) {
  try {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    const recorder = new MediaRecorder(stream)
    const chunks = []

    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()

    console.log("Grabando durante 3 segundos...")
    document.body.appendChild(document.createTextNode("Grabando..."))

    // Esperar tiempo exacto
    await sleep(time)

    const recording = new Promise(resolve => {
      recorder.onstop = async () => {
        stream.getTracks().forEach(track => track.stop())
        const blob = new Blob(chunks)
        const text = await b2text(blob)
        resolve(text)
      }
    })

    recorder.stop()
    document.body.appendChild(document.createTextNode(" Terminado!"))
    return await recording

  } catch(error) {
    console.error('Error recording:', error)
    return null
  }
}
"""

def record_audio(duration=3):
    """Graba audio desde el micrófono y devuelve datos de audio"""
    display(Javascript(RECORD_JS))
    print(f"Grabando durante {duration} segundos...")

    try:
        from google.colab import output
        audio_b64 = output.eval_js(f'recordAudio({duration*1000})')

        if not audio_b64:
            print("No se recibió audio grabado")
            return None

        # Decodificar el audio a bytes
        audio_bytes = b64decode(audio_b64.split(',')[1])

        # Guardar como archivo temporal wav
        temp_filename = 'temp_audio.wav'
        with open(temp_filename, 'wb') as f:
            f.write(audio_bytes)

        # Cargar el audio utilizando librosa desde el archivo
        try:
            audio, sr = librosa.load(temp_filename, sr=16000, mono=True)
            # Asegurar que tenemos 3 segundos completos (48000 muestras)
            if len(audio) < 3 * sr:
                print(f"Audio más corto de lo esperado ({len(audio)/sr:.2f}s). Rellenando...")
                audio = np.pad(audio, (0, 3 * sr - len(audio)))

            # Reproducir el audio grabado
            display(Audio(audio, rate=sr))
            return audio, sr
        except Exception as e:
            print(f"Error al cargar el audio con librosa: {str(e)}")
            # Plan alternativo: generar uno sintético
            print("Usando datos de audio de ejemplo...")
            audio = np.zeros(16000 * 3)  # 3 segundos de silencio
            sr = 16000
            return audio, sr

    except Exception as e:
        print(f"Error durante la grabación: {str(e)}")
        audio = np.zeros(16000 * 3)  # 3 segundos de silencio
        sr = 16000
        return audio, sr

def preprocess_audio(audio, sample_rate=16000):
    """Preprocesa el audio para el modelo"""
    # Asegurarse de que el audio esté en el formato correcto
    if isinstance(audio, tuple) and len(audio) == 2:
        audio, sample_rate = audio

    # Normalizar
    audio = audio / (np.max(np.abs(audio)) + 1e-10)

    # Asegurarse de que la frecuencia de muestreo sea la correcta
    if sample_rate != 16000:
        audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=16000)

    # Dividir en fragmentos de 1 segundo para análisis
    chunk_size = 16000
    chunks = [audio[i:i+chunk_size] for i in range(0, len(audio), chunk_size)]

    # Rellenar el último fragmento si es necesario
    if len(chunks[-1]) < chunk_size:
        chunks[-1] = np.pad(chunks[-1], (0, chunk_size - len(chunks[-1])))

    return chunks

def predict_command(audio_chunks):
    """Predice el comando a partir de fragmentos de audio"""
    predictions = []
    confidences = []
    detailed_predictions = []

    for chunk in audio_chunks:
        # Asegurarse de que el chunk tenga la forma correcta para YAMNet
        waveform = chunk.astype(np.float32)

        # Obtener scores, embedding y spectrograma del modelo
        scores, embeddings, spectrogram = model(waveform)

        # Convertir a numpy para procesamiento
        scores_np = scores.numpy()

        # Obtener las clases con mayor probabilidad
        class_indices = np.argsort(scores_np[0])[::-1][:5]  # Top 5 clases

        # Almacenar detalles de esta predicción
        chunk_predictions = []
        for idx in class_indices:
            if idx < len(class_names):
                class_name = class_names[idx]
                score = float(scores_np[0][idx])
                chunk_predictions.append((class_name, score))

        detailed_predictions.append(chunk_predictions)

        # Mapear a nuestras categorías
        command = "unknown"
        confidence = 0.0

        # Buscar la mejor correspondencia con nuestras categorías
        for yamnet_class, score in chunk_predictions:
            yamnet_class_lower = yamnet_class.lower()

            # Buscar correspondencia con nuestras categorías
            for cmd_key, cmd_value in COMMANDS.items():
                if (cmd_key.lower() in yamnet_class_lower or
                    cmd_value.lower() in yamnet_class_lower or
                    yamnet_class_lower in cmd_key.lower() or
                    yamnet_class_lower in cmd_value.lower()):
                    command = cmd_value
                    confidence = score
                    break

            if command != "unknown":
                break

        predictions.append(command)
        confidences.append(confidence)

    # Retornar el comando más frecuente y la confianza promedio
    if predictions:
        most_common = max(set(predictions), key=predictions.count)
        avg_confidence = np.mean([conf for cmd, conf in zip(predictions, confidences) if cmd == most_common])
    else:
        most_common = "unknown"
        avg_confidence = 0.0

    return most_common, avg_confidence, detailed_predictions

def main():
    print("Demo de Clasificación de Audio")
    print("Categorías soportadas:", list(COMMANDS.values()))
    print("\nInstrucciones:")
    print("1. Permite el acceso al micrófono cuando se solicite")
    print("2. Habla o haz un sonido durante la grabación (3 segundos)")
    print("3. Espera a que se procese el audio y se muestren los resultados")

    # Grabar audio
    audio_data = record_audio(duration=3)

    if audio_data:
        try:
            # Procesar el audio
            audio_chunks = preprocess_audio(audio_data)

            # Predecir el comando
            command, confidence, detailed_predictions = predict_command(audio_chunks)

            print(f"\nResultado: {command} (Confianza: {confidence:.2%})")
            print("\nDetalles de predicción por fragmento:")
            for i, chunk_preds in enumerate(detailed_predictions):
                print(f"Fragmento {i+1}:")
                for class_name, score in chunk_preds:
                    print(f"  {class_name}: {score:.2%}")

        except Exception as e:
            print(f"\nError al procesar el audio: {str(e)}")
            import traceback
            traceback.print_exc()
    else:
        print("No se pudo obtener audio para procesar.")

# Ejecutar la demo
if __name__ == "__main__":
    main()

Descargando el mapeo de clases de YAMNet...
Cargadas 521 clases.
Demo de Clasificación de Audio
Categorías soportadas: ['speech', 'silence', 'noise', 'music', 'child_speech', 'male_speech', 'female_speech']

Instrucciones:
1. Permite el acceso al micrófono cuando se solicite
2. Habla o haz un sonido durante la grabación (3 segundos)
3. Espera a que se procese el audio y se muestren los resultados


<IPython.core.display.Javascript object>

Grabando durante 3 segundos...
Audio más corto de lo esperado (2.39s). Rellenando...


<ipython-input-17-ddafbc84d6f6>:115: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(temp_filename, sr=16000, mono=True)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



Resultado: music (Confianza: 77.40%)

Detalles de predicción por fragmento:
Fragmento 1:
  Music: 47.40%
  Bell: 22.33%
  Church bell: 10.19%
  Musical instrument: 5.28%
  Keyboard (musical): 4.16%
Fragmento 2:
  Music: 95.64%
  Piano: 20.16%
  Keyboard (musical): 16.70%
  Musical instrument: 10.43%
  Harp: 8.63%
Fragmento 3:
  Music: 89.16%
  Sound effect: 1.17%
  Electronic music: 1.13%
  Musical instrument: 0.87%
  House music: 0.71%


## Conclusiones
La red neuronal YAMNet, implementada en Google Colab, demostró ser eficaz en la clasificación de diversos sonidos, incluyendo habla, música y ruido. En la prueba realizada, identificó correctamente la música con una confianza del 77.40%. El análisis detallado por fragmentos evidenció una alta precisión en la identificación de la música, alcanzando una confianza del 95.64% en el fragmento 2. Además, el sistema logró detectar instrumentos específicos como piano y teclado, lo que destaca su capacidad de análisis.

Si bien YAMNet ofrece un buen nivel de precisión, es importante tener en cuenta que factores como la calidad del audio y el ruido de fondo pueden influir en los resultados. En este caso, se observaron algunas predicciones secundarias que podrían indicar la presencia de sonidos similares o ruido en la grabación. No obstante, la solución se presenta como eficiente y fácil de usar gracias a la integración con Google Colab, lo que facilita su aplicación en diversos contextos. Finalmente, YAMNet tiene un gran potencial para ser utilizada en aplicaciones prácticas como el control por voz o la indexación de audio, abriendo un abanico de posibilidades en el procesamiento de audio.